In [1]:
import numpy as np

Importing phi and phi_test from train and test datasets

In [3]:
phi = np.loadtxt('data/train.csv', dtype='float', delimiter=',', skiprows=1,
                 usecols=tuple(range(1, 14)))

phi_test = np.loadtxt('data/test.csv', dtype='float', delimiter=',',
                      skiprows=1, usecols=tuple(range(1, 14)))


Importing y from train data set

In [4]:
y = np.loadtxt('data/train.csv', dtype='float', delimiter=',', skiprows=1,
               usecols=14, ndmin=2)

Concatenating coloumn of 1s to right of phi and phi_test

In [5]:
phi_test = np.concatenate((phi_test, np.ones((105, 1))), axis=1)

phi = np.concatenate((phi, np.ones((400, 1))), axis=1)

In [10]:
# phi_test.shape
# phi.shape

(400, 14)

**Feature Engineering**

Min max scaling on each coloumn of phi and phi_test

In [11]:
for x in range(0, 13):
    col_max = max(phi[:, x])
    col_min = min(phi[:, x])
    phi[:, x] = (phi[:, x] - col_min) / (col_max - col_min)
    phi_test[:, x] = (phi_test[:, x] - col_min) / (col_max - col_min)

Applying log scaling on y

In [12]:
y = np.log(y)

Function to calculate change in error function based on phi, w and p norm

In [16]:
def delta_w(p, phi, w):
    if p == 2:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lamb * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lamb * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return deltaw

Dictionary containing filenames as keys and p as values


In [14]:
file_names = {'output.csv': 2.0,
             'output_p1.csv': 1.75,
             'output_p2.csv': 1.5,
             'output_p3.csv': 1.3
             }

**For each item in this dictionary**

In [19]:
for (fname, p) in file_names.items():

    # Set initial w to zeros
    w = np.zeros((14, 1))

    # Hyperparameter lambda value (trail and error)
    lambd = 0.2

    # Maximum step size
    t = 0.00012

    # Calculate new value of w
    w_new = w - t * delta_w(p, phi, w)

    i = 0

    # Repeat steps until error between consecutive w is less than threshold
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * delta_w(p, phi, w)
        i = i + 1

    # Load values of id from the data file
    id_test = np.loadtxt('data/test.csv', dtype='int', delimiter=',',
                         skiprows=1, usecols=0, ndmin=2)

    # Calculate y for test data using phi test and applying inverse log
    y_test = np.exp(np.dot(phi_test, w_new))

    # Save the ids and y according to filename from dictionary
    np.savetxt(fname, np.concatenate((id_test, y_test), axis=1),
               delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')